The MITBIH dataset is constituted of 109446 beats, labeled with 5 different classes :

'N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4

N : Non-ecotic beats (normal beat)

S : Supraventricular ectopic beats

V : Ventricular ectopic beats

F : Fusion Beats

Q : Unknown Beats

The PTBHB dataset is constituted of 14552 beats, labeled with two different classes :

'0' for normal beat '1' for abnormal beat (Myocardial infarction)

All the beats are recorded with 187 points. The shorter beats are padded with zeros to reach 187.

The given notebook creates a model to predict the various classes of heartbeats based on MITBIH Dataset 

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import StratifiedKFold 
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D,GlobalAveragePooling1D
from tensorflow.keras import models, layers,Sequential,regularizers
from tensorflow.keras.utils import to_categorical

 ## Uploading dataset

In [ ]:
def dataset_creation(train_loc, test_loc):
    data_test = pd.read_csv(test_loc)
    data_train = pd.read_csv(train_loc)
    data_test.columns = [x for x in range(188)]
    data_train.columns = [x for x in range(188)]
    dataset = pd.concat([data_train, data_test], axis=0).sort_index()
    return dataset

## **Stratified K-Fold**

In [ ]:
def stratkfold(dataset):
    dataset["kfold"] = -1
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    X= dataset.iloc[:,:-1]
    y= dataset.iloc[:,-1]
    skf = StratifiedKFold(n_splits=5)
    for f, (t_, v_) in enumerate(skf.split(X,y)):
        dataset.loc[v_, 'kfold'] = f
    return dataset

## **Train-Test Split**

In [ ]:
def traintest_split(dataset):
    train=dataset[dataset["kfold"]!=4]
    valid=dataset[dataset["kfold"]==4]
    X_train=np.asarray(train.iloc[:,:187].values)
    y_train=train.iloc[:,187].values
    X_valid=np.asarray(valid.iloc[:,:187].values)
    y_valid=valid.iloc[:,187].values
    y_train=to_categorical(y_train)
    y_valid=to_categorical(y_valid)
    X_train=tf.expand_dims(X_train, axis=2)
    X_valid=tf.expand_dims(X_valid, axis=2)
    print(X_train.shape)
    return X_train, X_valid, y_train, y_valid

## **1-D CNN Model** 

In [ ]:
def make_model(X_train):
    model= Sequential()
    model.add(Convolution1D(32,5,activation='relu',input_shape=(X_train.shape[1],1)))
    model.add(Convolution1D(64,5,activation='relu'))         
    model.add(MaxPooling1D(3))
    model.add(Convolution1D(128, 3, activation='relu'))
    model.add(Convolution1D(256, 3, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(5,activation='softmax'))
    
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])
    return model

In [ ]:
train_loc = "../input/heartbeat/mitbih_train.csv"
test_loc= "../input/heartbeat/mitbih_test.csv"
dataset= dataset_creation(train_loc,test_loc)
data= stratkfold(dataset)
X_train,X_valid,y_train, y_valid = traintest_split(data)

In [ ]:
model=make_model(X_train)
history = model.fit(
    X_train,
    y_train,
    validation_split=0.1,
    batch_size=64,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
model.evaluate(X_valid,y_valid)

Training accuracy of 98.77% and test accuracy of 98.44% has been achieved from the given model 